In [13]:
import cv2
import numpy as np
import math

In [14]:
class ObjectDetection:
    def __init__(self, weights_path="/home/redwan/Desktop/Object detection/edit/yolov4.weights", cfg_path="/home/redwan/Desktop/Object detection/edit/yolov4.cfg"):
        print("Loading Object Detection")
        print("Running opencv dnn with YOLOv4")
        self.nmsThreshold = 0.4
        self.confThreshold = 0.5
        self.image_size = 608

        # Load Network
        net = cv2.dnn.readNet(weights_path, cfg_path)

        # Enable GPU CUDA
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
        self.model = cv2.dnn_DetectionModel(net)

        self.classes = []
        self.load_class_names()
        self.colors = np.random.uniform(0, 255, size=(80, 3))

        self.model.setInputParams(size=(self.image_size, self.image_size), scale=1/255)

    def load_class_names(self, classes_path="/home/redwan/Desktop/Object detection/edit/classes.txt"):

        with open(classes_path, "r") as file_object:
            for class_name in file_object.readlines():
                class_name = class_name.strip()
                self.classes.append(class_name)

        self.colors = np.random.uniform(0, 255, size=(80, 3))
        return self.classes

    def detect(self, frame):
        return self.model.detect(frame, nmsThreshold=self.nmsThreshold, confThreshold=self.confThreshold)

In [15]:
#from object_detection import ObjectDetection

od=ObjectDetection()

Loading Object Detection
Running opencv dnn with YOLOv4


In [16]:
cap=cv2.VideoCapture(0)
count =0
copy_of_prv_center_points=[]
tracking_objects={}
tracking_objects_copy={}
track_id=0

In [17]:
while True:
    ret,frame=cap.read()
    count+=1
    
    if not ret:
        break
    #center points of current frame   
    center_points=[]
        
    (class_ids, scores, boxes) = od.detect(frame)
    
    
    for box in boxes:
        (x,y,w,h)=box
        cx=int(0.5*(x+(x+w)))
        cy=int(0.5*(y+(y+h)))
        center_points.append((cx,cy))
        #print("FRAME no**",count," ",x,y,w,h)
        #cv2.circle(frame,(cx,cy),0,(0,0,255),-1)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),3)
    #beginning of the current and previous points frame   
    if count<=2:    
        for pt in center_points:
            for pt2 in copy_of_prv_center_points:
                distance=math.hypot(pt2[0]-pt[0],pt2[1]-pt[1])
                if distance<20:
                    tracking_objects[track_id]= pt
                    track_id+=1
            
    else:
        tracking_objects_copy=tracking_objects.copy()
        center_points_current_frame_copy=center_points.copy()
        for object_id,pt2 in tracking_objects_copy.items():
            object_exists=False
            for pt in center_points_current_frame_copy:
                distance=math.hypot(pt2[0]-pt[0],pt2[1]-pt[1])
                #update the ids position
                if distance<20:
                    tracking_objects[object_id]= pt
                    object_exists=True
                    if pt in center_points:
                        center_points_current_frame_copy.remove(pt)
                        continue
            #remove the lost id 
            if not object_exists:
                tracking_objects.pop(object_id)
        
        #Add new id found
        for pt in center_points:
            tracking_objects[track_id]=pt
            track_id+=1
            
    
    
                
        #cv2.circle(frame,(cx,cy),0,(0,0,255),-1)
    for object_id,pt in tracking_objects.items():
        cv2.circle(frame,pt,0,(0,0,255),-1)
        cv2.putText(frame,str(object_id),(pt[0],pt[1]-7),0,1,(0,0,255),3)
        
    #print("Tracking Objects:",tracking_objects)
        

    cv2.imshow('Frame',frame)
    #print("Current Frame:",center_points)
    #print("Previous Frame:",copy_of_prv_center_points)
    
    #make a copy of previous points
    copy_of_prv_center_points=center_points.copy()
    
    key=cv2.waitKey(1)
    if key==ord('n'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0] global /tmp/pip-req-build-driubr42/opencv/modules/dnn/src/dnn.cpp (1447) setUpNet DNN module was not built with CUDA backend; switching to CPU
